# Channel Map Plotting Demo

This notebook demonstrates the `cleaning-tools` package for creating channel maps with:
- **Sigma contours** (3σ, 5σ, etc.) with CARTA-style smoothing
- **Keplerian mask overlays** (optional)

## Features

1. **CARTA-style smoothing**: Gaussian smoothing (default sigma=1.7 pixels) before calculating contours
2. **Robust RMS calculation**: Uses iterative sigma clipping like CARTA
3. **Keplerian mask support**: Overlay mask contours to show model expectations
4. **Flexible visualization**: Control zoom, colormap, contour levels, etc.

In [ ]:
import sys
import os

# Add package to path if running from examples directory
sys.path.insert(0, os.path.abspath('..'))

from cleaning_tools import ChannelMapPlotter
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Example 1: Basic Usage (No Keplerian Mask)

Plot channel maps with sigma contours only.

In [ ]:
# Replace with your data file path
cube_path = '/Users/jea/AATau/cleaning_tests/weighting_taper_fits/AATau_N2H+_clean1_contsub_natural_notaper.fits'

# Create plotter (no Keplerian mask)
plotter = ChannelMapPlotter(cube_path=cube_path)

# Get cube info
info = plotter.get_info()
print(f"Cube shape: {info['shape']}")
print(f"Velocity range: {info['velocity_range'][0]:.2f} to {info['velocity_range'][1]:.2f} km/s")
print(f"Number of channels: {info['n_channels']}")

In [ ]:
# Plot channel maps with 3σ and 5σ contours
plotter.plot_channel_maps(
    start_channel=20,
    end_channel=45,
    sigma_levels=[3, 5, 10],  # Contour levels in sigma
    smooth_sigma=1.7,         # CARTA-style smoothing (FWHM=4 pixels)
    zoom_size=100,            # Zoom to central 200x200 pixels
    channels_per_page=20      # 20 channels per figure
)

## Example 2: With Keplerian Mask

Overlay Keplerian mask contours to compare with data.

In [ ]:
# Replace with your data and mask file paths
cube_path = '/Users/jea/AATau/cleaning_data/fits_2013SG2/AATau_N2H+_clean1_contsub.fits'
mask_path = '/Users/jea/AATau/cleaning_data/fits_2013SG2/AATau_N2H+_clean0_contsub.mask.fits'

# Create plotter with Keplerian mask
plotter_with_mask = ChannelMapPlotter(
    cube_path=cube_path,
    keplerian_mask_path=mask_path
)

info = plotter_with_mask.get_info()
print(f"Has Keplerian mask: {info['has_keplerian_mask']}")

In [ ]:
# Plot with both sigma contours and Keplerian mask
plotter_with_mask.plot_channel_maps(
    start_channel=30,
    end_channel=60,
    sigma_levels=[3, 5, 10, 20],  # Multiple sigma levels
    smooth_sigma=1.7,              # CARTA-style smoothing
    show_keplerian_mask=True,      # Show Keplerian mask contours
    zoom_size=100,
    keplerian_color='lime',        # Mask contour color
    sigma_color='black',           # Sigma contour color
    channels_per_page=20
)

## Example 3: No Smoothing (Native Resolution)

Plot at native resolution without smoothing.

In [ ]:
# Plot without smoothing
plotter.plot_channel_maps(
    start_channel=20,
    end_channel=45,
    sigma_levels=[3, 5],
    smooth_sigma=0,           # No smoothing
    zoom_size=100,
    channels_per_page=20
)

## Example 4: Customized Visualization

Full control over visualization parameters.

In [ ]:
# Custom visualization
plotter_with_mask.plot_channel_maps(
    start_channel=35,
    end_channel=45,
    sigma_levels=[3, 5, 10, 15, 20, 30],  # Many contour levels
    smooth_sigma=2.5,                      # More smoothing
    show_keplerian_mask=True,
    zoom_size=150,                         # Larger zoom
    channels_per_page=12,                  # Fewer channels per page
    figsize_per_row=4,                     # Taller figures
    cmap='viridis',                        # Different colormap
    vmin_sigma=-3,                         # Color range
    vmax_sigma=15,
    keplerian_color='cyan',                # Different mask color
    keplerian_linewidth=2.0,
    sigma_color='red',                     # Different contour color
    sigma_linewidth=1.0
)

## Example 5: Full Image (No Zoom)

Display entire field of view.

In [ ]:
# Full image
plotter.plot_channel_maps(
    start_channel=25,
    end_channel=40,
    sigma_levels=[3, 5, 10],
    smooth_sigma=1.7,
    zoom_size=None,           # No zoom - show full image
    channels_per_page=15
)

## Understanding the Output

### Channel Title Colors
- **Green**: Peak > 5σ (robust detection)
- **Orange**: Peak 3-5σ (marginal detection)
- **Red**: Peak < 3σ (likely noise)

### Contours
- **Black** (default): Sigma contours showing emission levels
- **Lime** (default): Keplerian mask showing expected emission regions
- **Yellow cross**: Image center (when zoomed)

### Smoothing vs. No Smoothing
- **With smoothing** (`smooth_sigma=1.7`):
  - Better SNR for extended emission
  - Lower RMS noise
  - Loss of angular resolution
  - Similar to CARTA display
  
- **Without smoothing** (`smooth_sigma=0`):
  - Native resolution
  - Higher RMS noise
  - Best for compact structures

### RMS Calculation
The RMS is calculated using **robust sigma clipping** from line-free channels:
1. Select edge channels (first & last 15 by default)
2. Iteratively clip outliers (3 iterations at 3σ)
3. Calculate standard deviation of clipped data

This matches CARTA's approach for noise estimation.

## Advanced: Manual RMS Calculation

You can manually specify line-free channels if needed.

In [ ]:
# Calculate RMS from specific channels
line_free_channels = list(range(0, 20)) + list(range(80, 100))
rms = plotter.calculate_rms(line_free_channels=line_free_channels)
print(f"RMS: {rms*1000:.3f} mJy/beam")
print(f"3σ: {3*rms*1000:.3f} mJy/beam")
print(f"5σ: {5*rms*1000:.3f} mJy/beam")

# With smoothing
rms_smooth = plotter.calculate_rms_smoothed(smooth_sigma=1.7,
                                            line_free_channels=line_free_channels)
print(f"\nRMS (smoothed): {rms_smooth*1000:.3f} mJy/beam")
print(f"Improvement factor: {rms/rms_smooth:.2f}x")

## Summary

The `ChannelMapPlotter` provides:

✅ **CARTA-style sigma contours** with optional smoothing  
✅ **Keplerian mask overlays** for model comparison  
✅ **Robust RMS calculation** using sigma clipping  
✅ **Flexible visualization** with zoom, colormaps, etc.  
✅ **SNR-based channel coloring** for quick assessment  

### Typical Workflow

1. Load cube with optional Keplerian mask
2. Plot channels with smoothing to identify emission
3. Adjust sigma levels and zoom as needed
4. Compare with Keplerian mask to assess model fit
5. Export figures for publications/presentations